In [30]:
import math
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import copy

In [31]:
# Embedding 
'''

'''

### SUBMODULES ###    
class PositionalEncoder(nn.Module):
    '''
    assume d_model = 512
    assume mas_seq_len = 80
    pe shape = [80, 512]
    '''
    def __init__(self, d_model, max_seq_len = 80): 
        super().__init__()
        self.d_model = d_model 
        # Positional Encoder
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (1000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (1000 ** ((2 * i)/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(1)
#         print(Variable(self.pe[:,:seq_len], requires_grad=False).cuda())
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
        return x
    
class MultiHeadAttention(nn.Module):
    '''
    q shap = [512, 512] ; [in_feature, out_feature]
    v shap = [512, 512] ; [in_feature, out_feature]
    k shap = [512, 512] ; [in_feature, out_feature]
    out = [512, 512] ; [in_feature, out_feature]
    '''
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()
        self.d_model = d_model 
        self.d_k = d_model // heads 
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
        
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        k = k.transpose(1,2)
        
        scores = self.attention(q, k, v, self.d_k, mask, self.dropout)
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
        output = self.out(concat)
        return output
    
    def attention(self, q, k, v, d_k, mask=None, dropout=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
        
        if mask is not None: 
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
        
        scores = F.softmax(scores, dim=-1)
        
        if dropout is not None: 
            scores = dropout(scores)
            
        output = torch.matmul(scores, v)
        return output
        
class FeedForward(nn.Module):
    """
    this is similar with dense layer or convolution layer
    """
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__() 
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x
        
class Norm(nn.Module): 
    '''
    norm = alpha*[(x - mean) / (std + eps)] + bias
    '''
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
        self.size = d_model
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
        
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm
        
        
        
        
### PUT IT ALL ABOVE TOGETHER ### 
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask): 
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2, x2, x2, mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        
        ''' might be like this
        x = self.Norm(x)
        x = x + self.dropout(self.attn(x2,x2,x2,mask))
        x = self.Norm(x)
        x = x + self.dropout(self.ff(x2))
        '''
        return x
    
class DecoderLayer(nn.Module): 
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Noem(d_model)
        self.norm_2 = Noem(d_model)
        self.norm_3 = Noem(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        
    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_1(self.attn_2(x2, e_outputs, e_outputs, trg_mask)) 
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x 
    
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

### ENCODER & DECODER ### 
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)
    
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N 
        print(vocab_size, d_model)
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N 
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(DecoderLayer(d_model, heads, dropt), N)
        self.norm = Norm(d_model)
        
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)
    
### Transformer ### 
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads, dropout):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, dropout)
        self.decoder = Decoder(trg_vocab, d_model, N, heads, dropout)
        self.out = nn.Linear(d_model, trg_vocab)
        
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output  = self.decoder(trg, e_outputs, src_mask, trg_mask)
        out = self.out(d_output)
        return output
            
        

def Model(src_vocab, trg_vocab):
    d_model = 512
    n_layers = 6
    heads = 8
    dropout = 0.1
    
    
    assert d_model % heads == 0
    assert dropout < 1

    model = Transformer(src_vocab, trg_vocab, d_model, n_layers, heads, dropout)
    model.to(device)
       
    if load_weights is not None:
        print("loading pretrained weights...")
        model.load_state_dict(torch.load(f'{load_weights}/model_weights'))
    else:
        for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
    
    return model


##### TESTING #####

import torch 
from torchtext import data
import numpy as np 
from torch.autograd import Variable

device = "cuda" if torch.cuda.is_available() else "cpu"

def nopeak_mask(size): 
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask = Variable(torch.from_numpy(np_mask == 0).to(device))
    return np_mask

def create_masks(src, trg, src_pad = True, trg_pad = True): 
    '''
    src_pad
    '''
    src_mask = (src != src_pad).unsqueeze(-2).to(device)
    
    if trg is not None: 
        trg_mask = (trg != trg_pad).unsqueeze(-2).to(device)
        size = trg.size(1)
        np_mask = nopeak_mask(size).to(device)
        trg_mask = trg_mask & np_mask
    else: 
        trg_mask = None
    
    return src_mask, trg_mask


In [35]:
d_model = 512
heads = 8
N = 6
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(FR_TEXT.vocab)
model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

NameError: name 'EN_TEXT' is not defined

In [ ]:
src = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]])
trg_input = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]])

src_mask, trg_mask = create_masks(src, trg_input)
    
# model(x, trg, src_mask, trg_mask)




model = Model(src, trg_input)

In [33]:
# pip install Tokenize

In [34]:
import pandas as pd
import torchtext
from torchtext import data
from Tokenize import tokenize
# from Batch import MyIterator, batch_size_fn
import os
import dill as pickle

def read_data(opt):
    
    if src_data is not None:
        try:
            src_data = open(src_data).read().strip().split('\n')
        except:
            print("error: '" + src_data + "' file not found")
            quit()
    
    if trg_data is not None:
        try:
            trg_data = open(trg_data).read().strip().split('\n')
        except:
            print("error: '" + trg_data + "' file not found")
            quit()
            
            
            
            
read_data(opt)
SRC, TRG = create_fields(opt)
opt.train = create_dataset(opt, SRC, TRG)
model = get_model(opt, len(SRC.vocab), len(TRG.vocab))

NameError: name 'opt' is not defined